In [1]:
import numpy as np
import pandas as pd
import cv2
import os

In [15]:
FREQ = 5

# Bulgarian Sign Language Detector #
## Data Collection ##
⚠️ <b>DON'T RUN </b>, THE VIDEOS ARE NOT IN THE REPO/ZIP (can be accessed through <a href='https://drive.google.com/drive/folders/101l61bluVN9f49zUGeR_AVPxk6osSpeX?usp=sharing'>this link</a> if needed)<br/><br/>
I could not find any dataset that contains Bulgarian sign language so I decided to create one. 
The raw sources are videos in which people are doing the signs. Every folder contains letter videos made by the same person (1 video = 1 letter; 1 folder = letters done by the same person). Most of them have used the same source to learn the signs from, however, even so, variations are present. One of the people (vid2) has used a different source and their signs vary more, espcially when it comes to angles.

Make sure you're at the root folder of the project 

In [29]:
pwd

'/home/avgustina/personal/AICourse/Machine Learning/Project/Final Project/Project'

### Creating data folder structure ###

Creating a data folder with a subfolder for each letter

In [17]:
rm -rf data

In [18]:
mkdir data

This feels barbaric but for the sake of reproducability:

In [19]:
bg_alphabet = ['A','B', 'V', 'G', 'D', 'E', 'J', 'Z', 'I', 'Ik', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'F', 'H', 'C', 'CH', 'SH', 'SHt', 'Y', 'Yrm', 'YU', 'YA']

In [20]:
for letter in bg_alphabet:
    os.makedirs('./data/' + letter)

In [21]:
pwd

'/home/avgustina/personal/AICourse/Machine Learning/Project/Final Project/Project'

### Populating the folders ###

In [22]:
video_folders = sorted(os.listdir('./videos'))

In [24]:
video_folders

['vid1', 'vid2', 'vid3', 'vid3.2', 'vid3.3', 'vid4', 'vid5', 'vid6']

A lot of videos start off with the person positioning their hand on screen before doing the sign - that's a lot of frames that would end up wrongly labelled. I mostly cut those parts with video editing tools, however, for some, I use video_snippets, which tell from which second to which second is the actual sign.

In [25]:
video_snippets = {
    'A': {'vid1': (1, 7), 'vid4': (1, 10)},
    'B': {'vid1': (0, 7), 'vid4': (1 ,7)},
    'V': {'vid1': (1, 6), 'vid4': (1 ,8)},
    'G': {'vid1': (1, 7), 'vid4': (1, 9)},
    'D': {'vid1': (2, 8), 'vid4': (1, 9)},
    'E': {'vid1': (2, 7), 'vid4': (2, 8)},
    'J': {'vid1': (1, 7), 'vid4': (1, 7)},
    'Z': {'vid1': (1, 7), 'vid4': (1, 7)},
    'I': {'vid1': (1, 7), 'vid4': (1, 7)},
    'Ik': {'vid1': (1, 18), 'vid4': (1, 5)},
    'K': {'vid1': (1, 7), 'vid4': (1, 8)},
    'L': {'vid1': (2, 5), 'vid4': (1, 8)},
    'M': {'vid1': (1, 7), 'vid4': (1, 9)},
    'N': {'vid1': (1, 7), 'vid4': (1, 8)},
    'O': {'vid1': (1, 7), 'vid4': (1, 8)},
    'P': {'vid1': (1, 6), 'vid4': (1, 8)},
    'R': {'vid1': (1, 5), 'vid4': (1, 10)},
    'S': {'vid1': (2, 8), 'vid4': (1, 9)},
    'T': {'vid1': (6, 11), 'vid4': (2, 8)},
    'U': {'vid1': (1, 7), 'vid4': (1, 8)},
    'F': {'vid1': (1, 7), 'vid4': (1, 8)},
    'H': {'vid1': (3, 9), 'vid4': (3, 8)},
    'C': {'vid1': (1, 8), 'vid4': (1, 8)},
    'CH': {'vid1': (1, 8), 'vid4': (1, 8)},
    'SH': {'vid1': (2, 8), 'vid4': (1, 8)},
    'SHt': {'vid1': (2, 10)}, 
    'Y': {'vid1': (2, 7), 'vid4': (1, 8)},
    'Yrm': {'vid1': (2, 12)}, 
    'YU': {'vid1': (0, 10), 'vid4': (1, 8)},
    'YA': {'vid1': (2, 8), 'vid4': (1, 8)},
}

In [26]:
def convert_sec_to_frames(vidcap, video_snippet):
    from_seconds, to_seconds = video_snippet
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    from_frame = int(fps * from_seconds)
    to_frame = int(fps * to_seconds)
    return from_frame, to_frame

def get_frames_from_video(video_folder_name, letter, target_path, frequency, video_snippet, file_extension = 'MOV'):  
    video_path = './videos/' + video_folder_name + '/'+ letter + '.' + file_extension
    vidcap = cv2.VideoCapture(video_path)
    
    # Check if valid path
    if not vidcap.isOpened():
        print(f"Error: Problem with opening video at path: {video_path}")
        return

    if video_snippet != None:
        from_frame, to_frame = convert_sec_to_frames(vidcap, video_snippet)
        # Move vidcap to that frame
        if from_frame: vidcap.set(cv2.CAP_PROP_POS_FRAMES, from_frame)
    else:
        from_frame, to_frame = 0, vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

    # Start reading frames
    success, image = vidcap.read()
    count = from_frame or 0
    pic_num = 0
    frames_folder_path = f'{target_path}/{video_folder_name}'
    os.makedirs(f'{target_path}/{video_folder_name}')
    while success and (to_frame and count <= to_frame):
      if (count % frequency == 0): 
          cv2.imwrite(f'{frames_folder_path}/{video_folder_name}{pic_num}.jpg', image)     # save frame as JPEG file
          pic_num += 1
          # print('Saved a new frame: ', pic_num)
          success, image = vidcap.read()
      success = vidcap.grab()
      count += 1
    print (f'Extracted {pic_num} frames from {video_path}')
    vidcap.release()

In [27]:
for video_folder_name in video_folders:
    # for letter_video in os.listdir(f'./videos/{video_folder_name}'):
    extension = os.listdir(f'./videos/{video_folder_name}')[0].split('.')[1]
    
    for letter in bg_alphabet:
        # get snippet if such
        if video_folder_name in video_snippets[letter]:
            snippet = video_snippets[letter][video_folder_name]
        else:
            snippet = None
        # print (extension)
        get_frames_from_video(
            video_folder_name, 
            letter,
            './data/' + letter,
            FREQ,
            snippet,
            extension
        )

Extracted 37 frames from ./videos/vid1/A.MOV
Extracted 36 frames from ./videos/vid1/B.MOV
Extracted 31 frames from ./videos/vid1/V.MOV
Extracted 37 frames from ./videos/vid1/G.MOV
Extracted 37 frames from ./videos/vid1/D.MOV
Extracted 31 frames from ./videos/vid1/E.MOV
Extracted 37 frames from ./videos/vid1/J.MOV
Extracted 37 frames from ./videos/vid1/Z.MOV
Extracted 37 frames from ./videos/vid1/I.MOV
Extracted 102 frames from ./videos/vid1/Ik.MOV
Extracted 37 frames from ./videos/vid1/K.MOV
Extracted 19 frames from ./videos/vid1/L.MOV
Extracted 37 frames from ./videos/vid1/M.MOV
Extracted 37 frames from ./videos/vid1/N.MOV
Extracted 37 frames from ./videos/vid1/O.MOV
Extracted 31 frames from ./videos/vid1/P.MOV
Extracted 25 frames from ./videos/vid1/R.MOV
Extracted 37 frames from ./videos/vid1/S.MOV
Extracted 31 frames from ./videos/vid1/T.MOV
Extracted 37 frames from ./videos/vid1/U.MOV
Extracted 37 frames from ./videos/vid1/F.MOV
Extracted 37 frames from ./videos/vid1/H.MOV
Extracte

In [28]:
os.system("tree -d")

.
├── data
│   ├── A
│   │   ├── vid1
│   │   ├── vid2
│   │   ├── vid3
│   │   ├── vid4
│   │   ├── vid5
│   │   └── vid6
│   ├── B
│   │   ├── vid1
│   │   ├── vid2
│   │   ├── vid3
│   │   ├── vid4
│   │   ├── vid5
│   │   └── vid6
│   ├── C
│   │   ├── vid1
│   │   ├── vid2
│   │   ├── vid4
│   │   ├── vid5
│   │   └── vid6
│   ├── CH
│   │   ├── vid1
│   │   ├── vid2
│   │   ├── vid3
│   │   ├── vid4
│   │   ├── vid5
│   │   └── vid6
│   ├── D
│   │   ├── vid1
│   │   ├── vid2
│   │   ├── vid3
│   │   ├── vid4
│   │   ├── vid5
│   │   └── vid6
│   ├── E
│   │   ├── vid1
│   │   ├── vid2
│   │   ├── vid4
│   │   ├── vid5
│   │   └── vid6
│   ├── F
│   │   ├── vid1
│   │   ├── vid2
│   │   ├── vid3
│   │   ├── vid4
│   │   ├── vid5
│   ���   └── vid6
│   ├── G
│   │   ├── vid1
│   │   ├── vid2
│   │   ├── vid3
│   │   ├── vid4
│   │   ├── vid5
│   │   └── vid6
│   ├── H
│   │   ├── vid1
│   │   ├── vid2
│   │   ├── vid3
│   │   ├── vid4
│   │   ├── vid5
│   │   └── vid6
│   ├── I
│ 

0